In [55]:
import sys
!{sys.executable} -m pip install mysql-connector-python
!{sys.executable} -m pip install nba_api
!{sys.executable} -m pip install beautifulsoup4



     ---------------------------------------- 0.0/143.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/143.0 kB ? eta -:--:--
     -- ------------------------------------- 10.2/143.0 kB ? eta -:--:--
     ---------- -------------------------- 41.0/143.0 kB 326.8 kB/s eta 0:00:01
     ----------------------- ------------- 92.2/143.0 kB 435.7 kB/s eta 0:00:01
     ---------------------------- ------- 112.6/143.0 kB 504.4 kB/s eta 0:00:01
     ------------------------------------ 143.0/143.0 kB 499.0 kB/s eta 0:00:00
  Obtaining dependency information for soupsieve>1.2 from https://files.pythonhosted.org/packages/4c/f3/038b302fdfbe3be7da016777069f26ceefe11a681055ea1f7817546508e3/soupsieve-2.5-py3-none-any.whl.metadata


In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
import mysql.connector as sql

conn = sql.connect(host=os.getenv("DB_HOST"),
                   database=os.getenv("DB_NAME"),
                   user=os.getenv("DB_USERNAME"),
                   password=os.getenv("DB_PASSWORD"),
                    )
mycursor = conn.cursor()

#mycursor.execute("CREATE TABLE customers (id INT AUTO_INCREMENT PRIMARY KEY, name VARCHAR(255), address VARCHAR(255))")
#mycursor.execute("DROP TABLE customers")
conn.close()

In [21]:
#from fp.fp import FreeProxy
proxy = FreeProxy(country_id=['US'], timeout=0.3, rand=True).get()
print(proxy)

http://23.238.33.186:80


In [90]:
#from nba_api.stats.endpoints import playercareerstats

# Nikola Jokić
career = playercareerstats.PlayerCareerStats(player_id='203999',proxy='http://186.234.124.127') 

ProxyError: HTTPSConnectionPool(host='stats.nba.com', port=443): Max retries exceeded with url: /stats/playercareerstats?LeagueID=&PerMode=Totals&PlayerID=203999 (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000012AD0C56CD0>, 'Connection to 186.234.124.127 timed out. (connect timeout=30)')))

In [43]:
import requests

# Define a list of proxy addresses and ports to test
#proxy_list = [FreeProxy(country_id=['US'], rand=True).get() for _ in range(10)]

# Create an empty dictionary to store working proxies
working_proxies = {}

# Define a URL to test the proxy (e.g., a website that you can access)
test_url = "https://www.google.com"

for proxy in proxy_list:
    try:
        # Create a session with the proxy
        session = requests.Session()
        session.proxies = {'http': proxy, 'https': proxy}

        # Make a test request using the proxy
        response = session.get(test_url)

        # Check if the request was successful (you can customize this check)
        if response.status_code == 200:
            working_proxies[proxy] = response.elapsed.total_seconds()
            print(f"Proxy {proxy} is working. Response time: {response.elapsed.total_seconds()} seconds.")
        else:
            print(f"Proxy {proxy} did not return a 200 OK response.")

    except Exception as e:
        print(f"Proxy {proxy} failed with the following error: {str(e)}")

# Print the working proxies
print("Working Proxies:")
for proxy, response_time in working_proxies.items():
    print(f"{proxy} (Response time: {response_time} seconds)")

# You can now use the working proxies in your application as needed.


Proxy 50.206.25.108:80 failed with the following error: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000012ACFD44250>, 'Connection to 50.206.25.108 timed out. (connect timeout=None)')))
Proxy 143.47.185.211:80 failed with the following error: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', OSError('Tunnel connection failed: 404 Not Found')))
Proxy 50.207.199.87:80 failed with the following error: HTTPSConnectionPool(host='www.google.com', port=443): Max retries exceeded with url: / (Caused by ProxyError('Unable to connect to proxy', ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000012ACFC86950>, 'Connection to 50.207.199.87 timed out. (connect timeout=None)')))
Proxy 50.206.25.106:80 failed with the following erro

KeyboardInterrupt: 

In [40]:
file_path = 'proxy.txt'

proxy_list = []
# Open the file for reading
with open(file_path, 'r') as file:
    # Iterate over each line in the file
    for line in file:
        # Process each line here
        proxy_list.append(line.strip())

In [42]:
proxy_list

['50.206.25.108:80',
 '143.47.185.211:80',
 '50.207.199.87:80',
 '50.206.25.106:80',
 '50.173.140.147:80',
 '107.1.93.208:80',
 '43.156.0.125:8888',
 '50.227.121.34:80',
 '172.232.60.102:8080',
 '50.218.57.65:80',
 '107.1.93.212:80',
 '50.168.163.176:80',
 '50.168.72.114:80',
 '50.231.110.26:80',
 '50.169.62.107:80',
 '107.1.93.223:80',
 '213.136.101.40:3128',
 '50.204.219.226:80',
 '85.8.68.2:80',
 '213.33.126.130:80',
 '116.203.206.255:80',
 '50.172.75.122:80',
 '93.188.161.84:80',
 '185.140.53.137:80',
 '207.2.120.53:80',
 '3.142.255.174:3128',
 '52.117.157.155:8002',
 '50.206.111.88:80',
 '50.168.163.181:80',
 '32.223.6.94:80',
 '50.204.233.30:80',
 '212.145.210.146:80',
 '50.171.32.224:80',
 '50.228.141.99:80',
 '50.168.210.238:80',
 '50.174.7.162:80',
 '20.157.194.61:80',
 '50.174.145.9:80',
 '50.171.32.222:80',
 '64.225.8.118:10000',
 '195.23.57.78:80',
 '50.228.141.98:80',
 '198.251.72.26:80',
 '100.19.135.109:80',
 '50.168.49.111:80',
 '50.220.168.134:80',
 '50.174.7.159:80',


In [63]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the webpage containing tables
url = 'https://free-proxy-list.net/'

# Send an HTTP GET request to the webpage
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the HTML content of the page using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all table elements on the webpage
    tables = soup.find_all('table')[0]

    if tables:
        # Create a list to store DataFrames for each table
        table_dataframes = []

        # Iterate through each table
        for table in tables:
            # Create an empty DataFrame for each table
            df = pd.DataFrame()

            # Iterate through rows and cells to extract table data
            for row in table.find_all('tr'):
                columns = row.find_all(['td', 'th'])  # Include both data cells and header cells
                if columns:
                    # Process and add data to the DataFrame
                    data = [col.get_text(strip=True) for col in columns]
                    df = pd.concat([df,pd.Series(data)])

            # Append the DataFrame for this table to the list
            table_dataframes.append(df)

        # Access and work with the DataFrames as needed
        for idx, df in enumerate(table_dataframes):
            print(f"Table {idx + 1}:")
            print(df)
            print("\n")

        # You can access individual DataFrames from the list as table_dataframes[0], table_dataframes[1], etc.

    else:
        print("No tables found on the webpage.")
else:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")


Table 1:
              0
0    IP Address
1          Port
2          Code
3       Country
4     Anonymity
..          ...
3      Cameroon
4     anonymous
5              
6           yes
7   30 mins ago

[2408 rows x 1 columns]


Table 2:
          0
0          
1      Date
2   Working
3     HTTPS
4       New
..      ...
4       27%
5         8
6        51
7        25
8      1010

[76 rows x 1 columns]


Table 3:
         0
0    Total
1     6604
2   Subnet
3     1345
4   Google
..     ...
7        6
8       TW
9        6
10  Others
11     127

[96 rows x 1 columns]




In [77]:
df = pd.DataFrame(table_dataframes[0].values.reshape(-1,8))
df.columns = df.iloc[0]

# Drop the first row (optional if you want to remove it)
df = df[1:].reset_index(drop=True)
df

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,167.99.236.14,80,US,United States,elite proxy,yes,no,7 secs ago
1,183.100.14.134,8000,KR,South Korea,elite proxy,yes,no,7 secs ago
2,102.134.98.222,8081,BI,Burundi,elite proxy,yes,no,7 secs ago
3,181.65.200.53,80,PE,Peru,elite proxy,yes,no,7 secs ago
4,133.242.229.79,33333,JP,Japan,elite proxy,yes,no,7 secs ago
...,...,...,...,...,...,...,...,...
295,110.238.111.229,8012,SG,Singapore,elite proxy,,yes,30 mins ago
296,201.95.254.137,3128,BR,Brazil,anonymous,,yes,30 mins ago
297,213.76.184.53,3128,PL,Poland,anonymous,no,yes,30 mins ago
298,35.240.219.50,8080,SG,Singapore,anonymous,no,yes,30 mins ago


In [66]:
table = soup.find_all(name='table', attrs={'xpath': '//*[@id="list"]/div/div[2]/div/table'})

<table class="table table-striped table-bordered"><thead><tr><th>IP Address</th><th>Port</th><th>Code</th><th class="hm">Country</th><th>Anonymity</th><th class="hm">Google</th><th class="hx">Https</th><th class="hm">Last Checked</th></tr></thead><tbody><tr><td>167.99.236.14</td><td>80</td><td>US</td><td class="hm">United States</td><td>elite proxy</td><td class="hm">yes</td><td class="hx">no</td><td class="hm">7 secs ago</td></tr><tr><td>183.100.14.134</td><td>8000</td><td>KR</td><td class="hm">South Korea</td><td>elite proxy</td><td class="hm">yes</td><td class="hx">no</td><td class="hm">7 secs ago</td></tr><tr><td>102.134.98.222</td><td>8081</td><td>BI</td><td class="hm">Burundi</td><td>elite proxy</td><td class="hm">yes</td><td class="hx">no</td><td class="hm">7 secs ago</td></tr><tr><td>181.65.200.53</td><td>80</td><td>PE</td><td class="hm">Peru</td><td>elite proxy</td><td class="hm">yes</td><td class="hx">no</td><td class="hm">7 secs ago</td></tr><tr><td>133.242.229.79</td><td>33

In [84]:
for _,data in pd.DataFrame(pd.Series([1,2,3,4]).values.reshape(2,2)).iterrows():
    print(data[1],data[0])

2 1
4 3


In [85]:
df = pd.DataFrame(pd.Series([1,2,3,4]).values.reshape(2,2))

In [87]:
df[df[0] != 1]

,0,1
1,3,4


In [89]:
'http://'+'1232'

'http://1232'